In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image
import pickle
from matplotlib import style
import time

style.use('ggplot')

In [2]:
size = 10
num_episodes = 25000
move_penalty = 1
food_reward = 50
enemy_penalty = 300
episode_rewards = []

In [3]:
epsilon = 0.9
eps_decay = 0.9998
show_every = 3000

In [4]:
start_q_table = None

In [5]:
learning_rate = 0.1
discount = 0.95

In [6]:
player_n = 1
food_n = 2
enemy_n =3

In [7]:
d = { 1:(255 , 175, 0),
      2:(0, 255, 0),
      3:(0, 0, 255) }

In [11]:
class Blob:
    def __init__(self):
        self.x = np.random.randint(0, size)
        self.y = np.random.randint(0, size)
    
    def __str__(self):
        return f"{self.x}, {self.y}"
    
    def __sub__(self, other):
        return (self.x - other.x, self.y - other.y)
    
    def action(self, choice):
        if choice == 0:
            self.move(x=1, y=1)  #to be left
        elif choice == 1:
            self.move(x=-1, y=1)  #to be up
        elif choice == 2:
            self.move(x=1, y=-1)  #to be right
        elif choice == 3:
            self.move(x=-1, y=-1) #to be down
        
    def move(self, x=False, y=False):
        if not x:
            self.x += 0
        else:
            self.x += x
        
        if not y:
            self.y += 0
        else:
            self.y += y
            
        if self.x < 0:
            self.x = 0
        elif self.x > size-1:
            self.x = size-1
            
        if self.y < 0:
            self.y = 0
        elif self.y > size-1:
            self.y = size-1

In [27]:
if start_q_table is None:
    q_table = {}
    
    for x1 in range(-size+1, size):
        for y1 in range(-size+1, size):
            for x2 in range(-size+1, size):
                for y2 in range(-size+1, size):
                    q_table[((x1, y1),(x2, y2))] = [np.random.uniform(-5,0) for i in range(4)]

else:
    with open(start__table, 'rb') as f:
        q_table = pickle.load(f)

In [21]:
for episode in range(num_episodes):
    player = Blob()
    food = Blob()
    enemy = Blob()
    
    if episode % show_every == 0:
        print(f"on # {episode}, epsilon:{epsilon}")
        print(f"{show_every} ep avg {np.mean(episode_rewards[-show_every:])}")
        show = Tru
    else:
        show = False
        
    episode_reward = 0
    for i in range(200):
        obs = (player-food, player-enemy)
        
        if np.random.random() > epsilon:
            action = np.argmax(q_table[obs])
        else:
            action = np.random.randint(0, 4)
            
        player.action(action)
        
        if player.x == enemy.x and player.y == enemy.y:
            reward = -enemy_penalty
        elif player.x == food.x and player.y == food.y:
            reward = food_reward    
        else:
            reward = -move_penalty
            
        new_obs = (player-food, player-enemy)
        max_future_q = np.max(q_table[new_obs])
        current_q = q_table[obs][action]
        
        if reward == food_reward:
            new_q = food_reward
        elif reward == -enemy_penalty:
            new_q = -enemy_penalty
        else:
            new_q = (1-learning_rate) * current_q + learning_rate * (reward + discount * max_future_q)
            
            
        q_table[obs][action] = new_q
        
        if show:
            env = np.zeros((size, size, 3), dtype=np.uint8)

            env[food.x][food.y] = d[food_n]  
            env[player.x][player.y] = d[player_n]  
            env[enemy.x][enemy.y] = d[enemy_n]
            
            img = Image.fromarray(env, 'RGB')  
            img = img.resize((300, 300)) 
            
            cv2.imshow('image', np.array(img))
            
            if reward == food_reward or reward == -enemy_penalty:
                if cv2.waitKey(1000) & 0xFFF == ord('q'):
                    break
            else:
                if cv2.waitKey(10) & 0xFFF == ord('q'):
                    break
        
        episode_reward += reward
        if reward == food_reward or reward == -enemy_penalty:
            break
    episode_rewards.append(episode_reward)
    epsilon *= eps_decay


moving_avg = np.convolve(episode_rewards, np.ones((show_every))/show_every, mode='valid')

plt.plot([i for i in range(len(moving_avg))], moving_avg)
plt.ylabel(f'reward {show_every}ma')
plt.xlabel('episode num')
plt.show()

with open(f"qtable-{int(time.time())}.pickle", "wb") as f:
    pickle.dump(q_table, f)

on # 0, epsilon:2.7478942614859856e-07
3000 ep avg 1.8993333333333333


IndexError: index 6 is out of bounds for axis 0 with size 2